In [ ]:
import pandas as pd
import config
l = 0
d = {}
filename = config.hny["frmpapname"]
with open(f"data/hny/{filename}") as fileobject:
    for line in fileobject:
        l += 1
        if "=" not in line:
            continue
        if l >= config.hny["startline"]:
            word, definition = line.split(config.hny["sep"], 1)
            word = word.strip()[1:]
            definition = definition.strip()
            d[word] = definition
df = pd.DataFrame.from_dict(d, orient="index", columns=["nl"])
df.index.name = "pap"
print(df)

In [ ]:
import re
# def extractFirstWord(definition):
def extractFirstWord(raw_pap, raw_nl):


    # raw_pap = definition["pap"]
    # raw_nl = definition["nl"]

    raw = raw_nl

    # if "!" in raw:
        # print(f"Removing ! from {raw}")
        # raw = re.sub("!", "", raw)
    # print(definition)
    without_tags = re.sub('[\(\[{].*?[\)\]}]', "", raw)
    without_num = re.sub('[0-9]\.', "", without_tags)
    # without_tags = re.sub('\[(.*?)\]', "", raw)
    # print(f"source:{definition}\n -tags:{without_tags}")

    # match = re.findall(r'^(.*?)[;|\.|,| *|!].*',without_tags)
    
    # match = re.findall(r'^(.*?)[;|\.|,| *|!]*?.*',without_tags)
    if re.findall(r'[;\.,!]+',without_num):
        # match = re.findall(r'([^0-9]+?)[;\.,\(].*',without_tags)
        match = re.findall(r'^(.*?)[;\.,\(].*',without_num)
        # match = re.findall(r'^(.*?)[;\.,\(].*',without_tags)
        # print(f"sign in {without_tags}")

        # print("without num", without_num)
        # print("picking first matching", match, without_tags)

        match = match[0]
    else:
        match = without_tags
    

    # match = re.findall(r'^(.*?)[;|\.|,| *].*',definition)
    # if not match:
    #     match = raw
    # else:
    #     match = match[0]
    # print(match)

    match = match.strip().lower()

    if not match.isalpha():
        print(f"Without tags was:{without_tags}")
        print(f"pap:{raw_pap}|cleaned:{match}|nl:{raw_nl} is not letters")

    return match


df = df.reset_index()
print(f"Length before dropping n/a: {len(df)}")
df['pap'].replace('', pd.np.nan, inplace=True)
df.dropna(subset=["pap"], inplace=True)
print(f"Length after dropping n/a: {len(df)}")

df["pap-simple"] = df.apply(lambda row: extractFirstWord(row["pap"], row["nl"]), axis=1)
# print(df.head(5))
df.to_csv("data/hny/pap-nl.csv")

In [ ]:
sentence = "ami ta bai kas"
sentence = "mi stima mi yu homber"
sentence = "mi ta kòre outo"
translation = ""
# print(df.loc["ta"])
# Try to ignore accents if the word isn't there
# Consider and extract synonyms
for word in sentence.split():
    word = word.lower()
    translation += df.loc[word]["simple"] + " "
print(f"source: {sentence} \n translation: {translation}")